In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%autosave 10
import numpy as np
from keras.layers.advanced_activations import SReLU
from keras import callbacks as ckbs
import random
import time
import gzip
from keras.models import model_from_json
import pickle as pkl
import keras
from keras.models import Sequential
from keras.layers.core import *
from keras.layers.wrappers import *
from keras.optimizers import *
from keras.utils import np_utils
from keras.layers.embeddings import Embedding
from keras.layers import LSTM
from docutils.languages.af import labels
from sklearn.model_selection import train_test_split
gpu_id='gpu0'
import os
os.environ["THEANO_FLAGS"] = "device="+gpu_id +",lib.cnmem=1"


Autosaving every 10 seconds


Using Theano backend.


In [2]:
def load_neural_network(file_from):
    (nn_arch, nn_weights_path) = pkl.load(open(file_from, 'rb'))
    nn = model_from_json(nn_arch)
    nn.set_weights(nn_weights_path)
    return nn

In [3]:
f = gzip.open('../data/processed/pkl_short/embeddings.pkl.gz', 'rb')
embeddings = pkl.load(f)
f.close()

label2Idx = embeddings['label2Idx']

#Inverse label mapping
idx2Label = {v: k for k, v in label2Idx.items()}

f = gzip.open('../data/processed/pkl_short/data.pkl.gz', 'rb')
test_data = pkl.load(f)
dev_data = pkl.load(f)
wiki_data = pkl.load(f)
f.close()

n_out = len(label2Idx)

array([ 879,  644,  277,   59,  918,  655,  262,   34,    0,    3,    0,
          3,  948,    0,    0,    0,    0,    0,    0,    0,    1,    1,
       1140,   -1,    0])

In [ ]:
np.max(wiki_data[0][0][:][:])

In [ ]:
int(nan)

In [5]:
class FullModelCheckpoint(ckbs.ModelCheckpoint):
    def on_epoch_end(self, epoch, logs={}):
        filepath = self.filepath.format(epoch=epoch, **logs)
        if self.save_best_only:
            current = logs.get(self.monitor)
            if current is None:
                warnings.warn('Can save best model only with %s available, '
                              'skipping.' % (self.monitor), RuntimeWarning)
            else:
                if self.monitor_op(current, self.best):
                    if self.verbose > 0:
                        print('Epoch %05d: %s improved from %0.5f to %0.5f,'
                              ' saving model to %s'
                              % (epoch, self.monitor, self.best,
                                 current, filepath))
                    self.best = current
                    pkl.dump([self.model.to_json(), self.model.get_weights()], open(filepath, 'wb'))
                else:
                    if self.verbose > 0:
                        print('Epoch %05d: %s did not improve' %
                              (epoch, self.monitor))
        else:
            if self.verbose > 0:
                print('Epoch %05d: saving model to %s' % (epoch, filepath))
            pkl.dump([self.model.to_json(), self.model.get_weights()], open(filepath, 'wb'))


In [6]:
def get_model(matrix):
    emb_layers = []
    for param_num in range(matrix.shape[2]):
        model = Sequential()
        embedding_max_index = np.max(matrix[:, :, param_num]) + 2
        out_size = 3
        if embedding_max_index > 300:
            out_size = 300
        elif embedding_max_index > 100:
            out_size = 10
        model.add(Embedding(embedding_max_index, out_size, input_length=matrix.shape[1]))
        emb_layers.append(model)
    
    merged = Merge(emb_layers, mode = 'concat')
    return merged
    

In [11]:
def get_data(data):
    print( "%d test size" % len(data))
    data = np.array(data)
    print(data.shape)
    X = [x[0] for x in data]
    y = [x[1] for x in data]
    return np.array(X), np.array(y)
    

In [12]:
X,y = get_data(dev_data)

1000 test size
(1000, 2)


In [15]:
merged = get_model(X)

In [16]:
model = Sequential()
model.add(merged)
model.add(LSTM(100, return_sequences=True, dropout_W=0.2))
model.add(LSTM(100, return_sequences=True, dropout_W=0.2))
model.add(LSTM(100, dropout_W=0.2))
model.add(Dropout(0.2))
model.add(Dense(100))
model.add(SReLU())
model.add(Dense(n_out, activation='softmax'))

sgd = SGD(lr=0.1, decay=1e-7, momentum=0.0, nesterov=False, clipvalue=3) 
rmsprop = RMSprop(clipvalue=3) 
nadam = keras.optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
model.compile(loss='sparse_categorical_crossentropy', optimizer=nadam,metrics=['accuracy', 'precision', 'recall', 'categorical_accuracy', 'fmeasure'])

model.summary()
checkpointer = FullModelCheckpoint(filepath= "../models/new_best_train_model", verbose=1, save_best_only=True)


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_26 (Embedding)         (None, 33, 300)       264000                                       
____________________________________________________________________________________________________
embedding_27 (Embedding)         (None, 33, 300)       193500                                       
____________________________________________________________________________________________________
embedding_28 (Embedding)         (None, 33, 10)        2780                                         
____________________________________________________________________________________________________
embedding_29 (Embedding)         (None, 33, 3)         180                                          
___________________________________________________________________________________________

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model.fit(X_train, y_train, batch_size=128, nb_epoch=100, validation_data=(X_test, y_test), callbacks=[checkpointer])
print('result')

Exception: The model expects 25 input arrays, but only received one array. Found: array with shape (800, 33, 25)

In [ ]:
model = load_neural_network('../models/best_train_model1')
pred = model.predict_classes(X, batch_size=128, verbose=1)

In [ ]:
classes = set(pred)
classes

In [ ]:
X = np.array(X)
test_data[0][0]
X[0]

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y, pred, classes))

In [ ]:
y= np.array(y)
print(y.shape)

In [ ]:
pred.shape

In [ ]:
print(classification_report(y, pred, classes))

In [ ]:
idx2Label[2]

In [ ]:
target_names = [idx2Label[0], idx2Label[1], idx2Label[2], idx2Label[3]]
print(classification_report(y, pred, target_names=target_names))